## OCR Portal web scrapper Py script
### 1st page web scrapper with normaliztion (not very efficient)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# create URL object
url = "https://ocrportal.hhs.gov/ocr/breach/breach_report.jsf"
page = requests.get(url)

In [ ]:
# parser-lxml = Change html to Python friendly format
# Obtain page's information
soup = BeautifulSoup(page.text, 'lxml')

In [ ]:
# Obtain info from tag <thead> for getting headers (from ocr portal, link above)
ocr_head = soup.find('thead')

In [ ]:
# Obtain every title of columns with tag <th>
headers = []
for i in ocr_head.find_all('th'):
 title = i.text
 headers.append(title)
 headers

In [ ]:
# Create a dataframe
ocr_data = pd.DataFrame(columns = headers)

In [ ]:
# Obtain info from tag <tbody> for getting body (from ocr portal, link above)
ocr_body = soup.find('tbody',id="ocrForm:reportResultTable_data")

In [ ]:
# Create a for loop to fill mydata
for j in ocr_body.find_all('tr'):
 row_data = j.find_all('td')
 row = [i.text for i in row_data]
 length = len(ocr_data)
 ocr_data.loc[length] = row

In [ ]:
# delete expand all column
ocr_data = ocr_data.drop('Expand All',axis=1)

In [ ]:
# add index column to data
index = [i for i in range(0,len(ocr_data))]
ocr_data['Index'] = index

In [ ]:
# bring index column to first position (rearrange columns)
columns_list = ocr_data.columns.to_list()
columns_list = columns_list[-1:] + columns_list[:-1] # last to first and then from there to last

In [ ]:
# merge columns in to df
# change column names (wihout spacing) for ease of writing sql queries
ocr_data = ocr_data[columns_list]
ocr_data.columns = ['ind','nameOfCoveredEntity','state','coveredEntityType','individualsAffected','breachSubmissionDate','typeOfBreach','locationOfBreachedInformation','businessAssociatePresent','webDescription']

In [ ]:
ocr_data

### Create SQL connection

#### Create Table query - 
    create table ocr_data(
	ind int primary key not null,
    nameOfCoveredEntity varchar(225),
    state varchar (225),
    coveredEntityType varchar(225),
    individualsAffected int,
    breachSubmissionDate varchar(225),
    typeOfBreach varchar(225),
    locationOfBreachedInformation varchar (225),
    businessAssociatePresent varchar(225),
    webDescription varchar(225)
);

In [ ]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='password',
                         db='medcurity')

# create cursor
cursor=connection.cursor()

data = ocr_data

In [ ]:
# # function to insert df into sql tables
# def sqlLoad(df):
#     cols = "`,`".join([str(i) for i in df.columns.tolist()])
#     for i,row in df.iterrows():
#         sql = "INSERT IGNORE INTO `table_name_in_sql` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
#         cursor.execute(sql, tuple(row))

#     # the connection is not autocommitted by default, so we must commit to save our changes
#     return connection.commit()


In [19]:
# add rows/tuples into table (append procees no overwrite)
# ocr_data.columns = ['ind','nameOfCoveredEntity','state','coveredEntityType','individualsAffected','breachSubmissionDate','typeOfBreach','locationOfBreachedInformation','businessAssociatePresent','webDescription']
cols = "`,`".join([str(i) for i in data.columns.tolist()])
for i,row in data.iterrows():
    sql = "CALL add_breach_data('{}', '{}', '{}', {}, '{}', '{}', '{}', '{}', '');".format(row.nameOfCoveredEntity.replace("'", "''"), row.state, row.coveredEntityType, int(row.individualsAffected), row.breachSubmissionDate, row.typeOfBreach, row.locationOfBreachedInformation, row.businessAssociatePresent)
    print(sql)
    cursor.execute(sql)

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

CALL add_breach_data('UHS of Delaware, Inc. ', 'PA', 'Business Associate', 40290, '03/29/2023', 'Hacking/IT Incident', 'Email', 'Yes', '');
CALL add_breach_data('American Pain and Wellness, PLLC', 'TX', 'Healthcare Provider', 7457, '03/24/2023', 'Hacking/IT Incident', 'Network Server', 'No', '');
CALL add_breach_data('NewBridge Services', 'NJ', 'Healthcare Provider', 1457, '03/24/2023', 'Hacking/IT Incident', 'Network Server', 'No', '');
CALL add_breach_data('Atlantic General Hospital', 'MD', 'Healthcare Provider', 26591, '03/24/2023', 'Hacking/IT Incident', 'Network Server', 'No', '');
CALL add_breach_data('Frideres Dental LLC', 'OR', 'Healthcare Provider', 1596, '03/24/2023', 'Hacking/IT Incident', 'Email', 'No', '');
CALL add_breach_data('New Medical Healthcare', 'KS', 'Healthcare Provider', 1557, '03/22/2023', 'Unauthorized Access/Disclosure', 'Email', 'No', '');
CALL add_breach_data('US Wellness Inc.', 'MD', 'Business Associate', 11459, '03/22/2023', 'Hacking/IT Incident', 'Networ

#### Resources
        https://www.dataquest.io/blog/sql-insert-tutorial/
        
        https://machinelearningprojects.net/schedule-a-python-script-in-aws-glue-as-a-job/

        https://medium.com/@orjadamucollari/data-pipeline-automation-with-mysql-python-and-aws-2b5cf34ffcf8

In [ ]:
state_df = ocr_data[['state']]
state_df = state_df.rename(columns={"state":"name"})
state_df.insert(0,'id',range(1,1+len(state_df)))

In [ ]:
# add rows/tuples into table state (append procees no overwrite)
cols = "`,`".join([str(i) for i in state_df.columns.tolist()])
for i,row in state_df.iterrows():
    sql = "INSERT IGNORE INTO `state` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
location_df = ocr_data[['locationOfBreachedInformation']]

In [ ]:
location_df = location_df.rename(columns={"locationOfBreachedInformation":"location"})
location_df.insert(0,'id',range(1,1+len(location_df)))

In [ ]:
# add rows/tuples into table breach_location (append procees no overwrite)
cols = "`,`".join([str(i) for i in location_df.columns.tolist()])
for i,row in location_df.iterrows():
    sql = "INSERT IGNORE INTO `breach_location` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
breach_type_df = ocr_data[['typeOfBreach']]

In [ ]:
breach_type_df = breach_type_df.rename(columns={"typeOfBreach":"name"})
breach_type_df.insert(0,'id',range(1,1+len(breach_type_df)))

In [ ]:
# add rows/tuples into table breach_location (append procees no overwrite)
cols = "`,`".join([str(i) for i in breach_type_df.columns.tolist()])
for i,row in breach_type_df.iterrows():
    sql = "INSERT IGNORE INTO `breach_type` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
business_entity_df = ocr_data[['nameOfCoveredEntity']]

In [ ]:
business_entity_df = business_entity_df.rename(columns={"nameOfCoveredEntity":"name"})
business_entity_df.insert(0,'id',range(1,1+len(business_entity_df)))

In [ ]:
# add rows/tuples into table buisness_entity (append procees no overwrite)
cols = "`,`".join([str(i) for i in business_entity_df.columns.tolist()])
for i,row in business_entity_df.iterrows():
    sql = "INSERT IGNORE INTO `buisness_entity` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
covered_entity_type_df = ocr_data[['coveredEntityType']]

In [ ]:
covered_entity_type_df = covered_entity_type_df.rename(columns={"coveredEntityType":"type"})
covered_entity_type_df.insert(0,'id',range(1,1+len(covered_entity_type_df)))

In [ ]:
# add rows/tuples into table covered_entity_type (append procees no overwrite)
cols = "`,`".join([str(i) for i in covered_entity_type_df.columns.tolist()])
for i,row in covered_entity_type_df.iterrows():
    sql = "INSERT IGNORE INTO `covered_entity_type` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()

In [ ]:
#individualsAffected, breachSubmissionDate,businessAssociatePresent, webDescription
test_final = ocr_data[['breachSubmissionDate','businessAssociatePresent','individualsAffected','webDescription']]
test_final.insert(0,'id',range(1,1+len(test_final)))

In [ ]:
# add id columns into final df
list_id_names = ['buisness_entity_id','state_id','entity_type_id','breach_type_id','location_of_breach_id']
list_ids = [business_entity_df[['id']], state_df[['id']], covered_entity_type_df[['id']], breach_type_df[['id']], location_df[['id']]]
for i in range(1,len(list_ids)+1):
    test_final.insert(i,list_id_names[i-1],list_ids[i-1])

In [ ]:
# rename some columns 
test_final.rename(columns = {'breachSubmissionDate':'breach_date', 'businessAssociatePresent':'business_associate_present',
                             'individualsAffected':'individuals_affected','webDescription':'web_description'}, inplace = True)


In [ ]:
# change data types to import successfully into sql
test_final['breach_date'] = pd.to_datetime(test_final['breach_date'])
test_final['business_associate_present'] = test_final.business_associate_present.map(dict(Yes=1, No=0))

In [ ]:
test_final

In [ ]:
# add rows/tuples into table state (append procees no overwrite)
cols = "`,`".join([str(i) for i in test_final.columns.tolist()])
for i,row in test_final.iterrows():
    sql = "INSERT INTO `breaches` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    cursor.execute(sql, tuple(row))

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit()